### Setup

In [1]:
// import dependencies
var {smock} = require('@defi-wonderland/smock')

var moment = require('moment')
var { constants } = require('../utils')
var { advanceTimeAndBlock } = require('../utils/evm')
var { evm, common, contracts } = require('../utils');
var { toUnit } = require('../utils/bn')
var { getLatestBlockTimestamp, getBlockTimestamp } = require('../utils/evm')
var { next, clear, bnToNumber } = require('../utils/jupyter')
var { Notebook } = require('../utils/notebook')
var { ethers } = require('hardhat')

var $ = new Notebook();

clear();

// console.log(smocklib.smock)

In [2]:
// setup tools and contracts

next(async () => {
    await $.setup(constants.FORK_BLOCK_NUMBER);
    
    provider = await $.impersonate(constants.RICH_DAI_ADDRESS);
    
    // contract artifacts
    ls = await $.fetch('LongShort', constants.LONG_SHORT_ADDRESS)
    synthF3Long = await $.fetch('SyntheticToken', constants.SYNTHETIC_TOKEN_ADDRESS_F3_LONG)
    synthF3Short = await $.fetch('SyntheticToken',constants.SYNTHETIC_TOKEN_ADDRESS_F3_SHORT)
    ym = await $.fetch('YieldManagerAave', constants.YIELD_MANAGER_ADDRESS)
    oracleManager = await $.fetch('OracleManagerFlippening_V0', constants.ORACLE_MANAGER_ADDRESS_F3)
    alphaFLT = await $.fetch('AlphaTestFLT',constants.ALPHA_FLT_ADDRESS)
    treasury = await $.fetch('TreasuryAlpha',constants.TREASURY_ALPHA_ADDRESS)
    
    dai = await $.fetch('ERC20', constants.DAI_ADDRESS )
    
    // fake contracts
    oracleManagerFake = await smock.fake('OracleManagerFlippening_V0');
    longShortFake = await smock.fake('LongShort');
});

In [37]:
//Checking total supply of long F3 
synthF3Long.totalSupply()

BigNumber { _hex: '0x29', _isBigNumber: true }

In [4]:
//Approving transaction for the user
dai.connect(provider).approve(ls.address,1000000000)

{
  hash: '0x9b88dadac9cc7bbfa51f0082a48d9345b8b4d2b1d9c907d249d31369e1ec9705',
  type: 0,
  accessList: null,
  blockHash: '0x77629442b052f4015aa19b2711715bea57d1470dc9fa357897a964db136b57d4',
  blockNumber: 20001019,
  transactionIndex: 0,
  confirmations: 1,
  from: '0x06959153B974D0D5fDfd87D561db6d8d4FA0bb0B',
  gasPrice: BigNumber { _hex: '0x01dcd65000', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01badb18', _isBigNumber: true },
  to: '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 3364,
  data: '0x095ea7b3000000000000000000000000168a5d1217aecd258b03018d5bf1a1677a07b733000000000000000000000000000000000000000000000000000000003b9aca00',
  r: '0x0000000000000000000000000000000000000000000000000000000000000001',
  s: '0x0000000000000000000000000000000000000000000000000000000000000002',
  v: 27,
  creates: null,
  chainId: 0,
  wait: [Function (anonymous)]
}

In [17]:
//Mint long next price for user in F3 market
ls.connect(provider).mintLongNextPrice(synthF3Long.marketIndex(),1000000)

{
  hash: '0xfb98a0d57a1c70ab0b0c423dd02b7dac33b29d89e1aac0dd2a1bbe0ed004389d',
  type: 0,
  accessList: null,
  blockHash: '0xe46ee216a91eec2e59c366c962627105dcd70a9e87f8b052f2fb4e7fd2e5df55',
  blockNumber: 20001028,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x01dcd65000', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bada98', _isBigNumber: true },
  to: '0x168a5d1217AEcd258b03018d5bF1A1677A07b733',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 780,
  data: '0x897a078600000000000000000000000006959153b974d0d5fdfd87d561db6d8d4fa0bb0b0000000000000000000000000000000000000000000000000000000000000001',
  r: '0xe38dc869eccb7c76852b0efc15ce7bac5d3f91f9ab2b694446debc9b0447ee42',
  s: '0x2acdbffe34e8bb6ddb4dc070da69f71800c4fe96a63516babc6182f8514cee97',
  v: 62709,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}

In [18]:
//Forcing an oracle price update for next price execution to go through
oracleManagerFake.updatePrice.returns('42')

In [ ]:
//Running next price execution for the user
ls.executeOutstandingNextPriceSettlementsUser(constants.RICH_DAI_ADDRESS,1)

In [16]:
oracleManagerFake.connect(provider).updatePrice()

{
  hash: '0x5bc5da792b396a6f7bf884ec02ea06cdf01b8aff0c8865aea704f146dcb04b4d',
  type: 0,
  accessList: null,
  blockHash: '0xec85e9740d48edcb73162221d2af696a4d0378ce29d03a7259c6725f67110244',
  blockNumber: 20001027,
  transactionIndex: 0,
  confirmations: 1,
  from: '0x06959153B974D0D5fDfd87D561db6d8d4FA0bb0B',
  gasPrice: BigNumber { _hex: '0x01dcd65000', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x4DF25aBf3Cd7A94B3063Aba8a339bFd25FA72207',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 3369,
  data: '0x673a7e28',
  r: '0x0000000000000000000000000000000000000000000000000000000000000001',
  s: '0x0000000000000000000000000000000000000000000000000000000000000002',
  v: 27,
  creates: null,
  chainId: 0,
  wait: [Function (anonymous)]
}

In [7]:
oracleManager.getLatestPrice()

BigNumber { _hex: '0x023d393ebf869096a4', _isBigNumber: true }

In [11]:
oracleManager.updatePrice()

{
  hash: '0xdbe3d17ab455e4184f4ede3b9f74a0192d2508a5a6bbd0d964538c4224eb205c',
  type: 0,
  accessList: null,
  blockHash: '0xdf729ef1100bf73bf660f5ae817e1842051c28fddd715aa7bae3b192eaaab4bd',
  blockNumber: 20001022,
  transactionIndex: 0,
  confirmations: 1,
  from: '0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266',
  gasPrice: BigNumber { _hex: '0x01dcd65000', _isBigNumber: true },
  gasLimit: BigNumber { _hex: '0x01bad458', _isBigNumber: true },
  to: '0x70a760ACD5503A8D6746BC7F00571F570ae0aD44',
  value: BigNumber { _hex: '0x00', _isBigNumber: true },
  nonce: 778,
  data: '0x673a7e28',
  r: '0x1dc64ef708a791304f06249c5b6a71927a31ad6b18648c9e51fde659f16c0eab',
  s: '0x042b94778e81812be26b0260018e13e48fdbce3be38e11b9f2e988923c29d745',
  v: 62709,
  creates: null,
  chainId: 31337,
  wait: [Function (anonymous)]
}

In [49]:
//Checking total supply of long F3 
synthF3Long.totalSupply()

In [32]:
//Checking total supply of alphaFLT
alphaFLT.totalSupply()

BigNumber { _hex: '0x0d1ec18243dc61f96d', _isBigNumber: true }

In [26]:
//Checking market value of long F3
ls.marketSideValueInPaymentToken(1,true)
0x18b78f7dfa2132fb774a / 10**18

116722.88944970035

In [23]:
//Checking market value of short F3
ls.marketSideValueInPaymentToken(1,false)
0x12a2debba8d1573adcb2 / 10**18

88007.01883890868

In [19]:
 (0x12a2def41f9b98800525 - 0x12af0cfc6353d5789fca) / 10**18

-224.67790446863268

In [50]:
treasury.paymentToken()

'0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063'

In [14]:
ls.marketUpdateIndex(1)

BigNumber { _hex: '0x0e45', _isBigNumber: true }

In [15]:
0x0e45

3653

In [24]:
longShortFake.marketUpdateIndex.returns(3690)

In [26]:
longShortFake.marketUpdateIndex(2)

BigNumber { _hex: '0x0e6a', _isBigNumber: true }

In [25]:
0x0e6a

3690